In [1]:
import numpy as np
import Pk_library as PKL
import os
import glob
from tqdm.notebook import tqdm

import sys
sys.path.append('../../py/')

from cosmoSim import cosmoSim

In [2]:
outdir = '../../data_prods/'
data_dir = '/home/ryan/Data'
subdir_list = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

print(subdir_list)

['/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma0.1', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9', '/home/ryan/Data/run_2cDM_L3N512_HY_power00_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1', '/home/ryan/Data/run_2cDM_L25N256_DM_power00_sigma1_dir_4_Vkick55.65', '/home/ryan/Data/run_CDM_L5N256_DM', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_9', '/home/ryan/Data/run_SIDM_L3N256_DM_power0_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L25N256_DM_power00_sigma1_dir_7_Vkick119.90', '/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_1', '/home/ryan/Data/lost+found', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma1_dir_5', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_3', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma10', '/home/ryan/Data/run_2

In [3]:
def calculate_pk(snap, out, part_types=[1], grid_size=256):  

    # parameters
    snapshot      = snap #snapshot name
    grid          = grid_size    #grid size
    particle_type = part_types #use dark matter [1]
    do_RSD        = False   #move particles to redshift-space and calculate Pk in redshift-space
    axis          = 1      #RSD placed along the y-axis
    cpus          = 8      #number of openmp threads
    folder_out    = out #folder where to write results
    # compute power spectrum of the snapshot

    PKL.Pk_Gadget(snapshot, grid, particle_type, do_RSD, axis, cpus, folder_out)



In [4]:
rewrite = False
for subdir in tqdm(subdir_list):
    
    #run name is just folder name
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue

    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')

    snaps = sorted(glob.glob(subdir + '/snap*'))
    
    if not rewrite:
        if (len(glob.glob(fpath + "/Pk_CDM_z=*")) == len(snaps)):
            print(f'PK files exist for {run_name}.')
            print('Skipping...')
            continue
          
        
    # fsize = os.stat(snaps[-1]).st_size / (1024**3)
    # if fsize > 20:
    #     print(f'File size too big for {run_name}...')
    #     continue
    '''
    Run pylians here
    '''

    run = cosmoSim(run_name)
    grid_size = run.npart
    print('Running pylians power spectra for ', run_name)
    for i, snap in enumerate(snaps):
        print(f"Working on {snap}")
        part_types = [1]
        if run.baryon_type == "HY":
            part_types = [0, 1, 4]
        try:
            calculate_pk(snap, fpath, part_types, grid_size)
        except Exception as e:
            print(e)
            if run.baryon_type == "HY":
                part_types = [0, 1]
            try:
                calculate_pk(snap, fpath, part_types, grid_size)
            except Exception as e:
                part_types = [1]
                calculate_pk(snap, fpath, part_types, grid_size)

    

  0%|          | 0/189 [00:00<?, ?it/s]

../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2 already exists!
PK files exist for run_2cDM_L3N256_DM_powerm2m2_sigma1_dir_2.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90 already exists!
PK files exist for run_2cDM_L3N256_DM_power00_sigma1_dir_7_Vkick119.90.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1 already exists!
PK files exist for run_2cDM_L3N256_DM_powerm2m2_sigma0.1.
Skipping...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9 already exists!
PK files exist for run_2cDM_L3N256_DM_powerm2m2_sigma5.0_dir_9.
Skipping...
../../data_prods/run_2cDM_L3N512_HY_power00_sigma1 already exists!
PK files exist for run_2cDM_L3N512_HY_power00_sigma1.
Skipping...
../../data_prods/run_2cDM_L3N256_HY_power00_sigma0.1 already exists!
PK files exist for run_2cDM_L3N256_HY_power00_sigma0.1.
Skipping...
../../data_prods/run_2cDM_L25N256_DM_power00_sigma1_dir_4_Vkick55.65 already exists!
PK files exist for run_2cDM_L25N2

/home/ryan/projects/Medvedev/dm_sims/jupyter/snaps_reqd/../../py/cosmoSim.py:57: UserWarning: Vkick not explicitly set in run run_2cDM_710_HY_powerm2m2_sigma1! Assuming 100 km/s...
  warnings.warn(f'Vkick not explicitly set in run {self.run_name}! Assuming 100 km/s...')


47.505 < X [Mpc/h] < 52.541
47.150 < Y [Mpc/h] < 52.482
47.653 < Z [Mpc/h] < 52.153


Computing power spectrum of the field...
Time to complete loop = 0.01
Time taken = 0.01 seconds
Working on /home/ryan/Data/run_2cDM_710_HY_powerm2m2_sigma1/snap_001.hdf5

READING SNAPSHOTS PROPERTIES
Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
47.495 < X [Mpc/h] < 52.543
47.142 < Y [Mpc/h] < 52.489
47.642 < Z [Mpc/h] < 52.155


Computing power spectrum of the field...
Time to complete loop = 0.01
Time taken = 0.01 seconds
Working on /home/ryan/Data/run_2cDM_710_HY_powerm2m2_sigma1/snap_002.hdf5

READING SNAPSHOTS PROPERTIES
Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
47.484 < X [Mpc/h] < 52.545
47.131 < Y [Mpc/h] < 52.498
47.629 < Z [Mpc/h] < 52.157


Computing power spectrum of the field...
Time to complete loop = 0.01
Time taken = 0.02 seconds
Working on /home/ryan/Data/run_2cDM_710_HY_powerm2m2_sigma1/snap_003

/home/ryan/projects/Medvedev/dm_sims/jupyter/snaps_reqd/../../py/cosmoSim.py:57: UserWarning: Vkick not explicitly set in run run_2cDM_710_HY_power00_sigma1! Assuming 100 km/s...
  warnings.warn(f'Vkick not explicitly set in run {self.run_name}! Assuming 100 km/s...')


Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
47.560 < X [Mpc/h] < 52.479
47.212 < Y [Mpc/h] < 52.409
47.688 < Z [Mpc/h] < 52.099


Computing power spectrum of the field...
Time to complete loop = 0.01
Time taken = 0.01 seconds
Working on /home/ryan/Data/run_2cDM_710_HY_power00_sigma1/snap_001.hdf5

READING SNAPSHOTS PROPERTIES
Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
47.553 < X [Mpc/h] < 52.478
47.208 < Y [Mpc/h] < 52.413
47.679 < Z [Mpc/h] < 52.099


Computing power spectrum of the field...
Time to complete loop = 0.01
Time taken = 0.02 seconds
Working on /home/ryan/Data/run_2cDM_710_HY_power00_sigma1/snap_002.hdf5

READING SNAPSHOTS PROPERTIES
Problem reading the block MASS

READING SNAPSHOTS PROPERTIES
Computing power spectrum...
47.545 < X [Mpc/h] < 52.477
47.202 < Y [Mpc/h] < 52.419
47.668 < Z [Mpc/h] < 52.098


Computing power spectrum of the field...
Time to complete loop = 0.01
Time take